In [ ]:
import os

import torch
import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

: 

In [37]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

dataset_name = 'UniversityPhysicsVolume1_textbook_info'

worksheet = gc.open(dataset_name).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
metadata_df = pd.DataFrame.from_records(data=rows[1:], columns=rows[0])

In [38]:
metadata_df.head()

,Chapter Name (Topic),Chapter Number,Subchapter Name (Subtopic),Subchapter Number,Subchapter Learning Objectives
0,Units and Measurement,1,The Scope and Scale of Physics,1.1,Describe the scope of physics.\nCalculate the ...
1,Units and Measurement,1,Units and Standards,1.2,Describe how SI base units are defined.\nDescr...
2,Units and Measurement,1,Unit Conversion,1.3,Use conversion factors to express the value of...
3,Units and Measurement,1,Dimensional Analysis,1.4,Find the dimensions of a mathematical expressi...
4,Units and Measurement,1,Estimates and Fermi Calculations,1.5,Estimate the values of physical quantities.


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
path = 'drive/Shareddrives/IDEAL Pedagogy/Learning Goal AI Tool/OpenStax Dataset/University Physics Volume 1/'

In [41]:
with open(path + '8.txt') as f:
  lines = [l.strip() for l in f]

lines[0]

'8.1 Potential Energy of a System'

In [42]:
import re

In [43]:
re.match('[0-9]\.[0-9]*', '82.1 hi')

In [44]:
def parse_question_file(filename, path=path):
  chapter_num = int(filename[:-4]) # everything before .txt extension
  with open(path + filename) as f:
    lines = [l.strip() for l in f]

  questions = {}
  question_nums = []
  current_subchapter = ''
  for line in lines:
    # when we encounter subchapter heading
    subchapter_num = re.match('[0-9]+\.[0-9]+', line)
    if subchapter_num:
      current_subchapter = subchapter_num.group(0)
      questions[current_subchapter] = []
      continue

    # when we encounter questions
    question_num = re.match('[0-9]+\. ', line)
    if question_num:
      question_num = question_num.group(0)
      questions[current_subchapter].append(line[len(question_num):])
      question_nums.append(question_num)
      continue
    
    # if this is part of a previous question
    questions[current_subchapter][-1] += line
  
  return questions, question_nums

In [45]:
questions, question_nums = {}, []
for filename in os.listdir(path):
  if filename.endswith('txt'):
    q, q_num = parse_question_file(filename)
    questions.update(q)
    question_nums.extend(q_num)

questions.keys()

dict_keys(['1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '2.1', '2.2', '2.3', '2.4', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '4.1', '4.2', '4.3', '4.4', '4.5', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '6.1', '6.2', '6.3', '6.4', '7.1', '7.2', '7.3', '7.4', '8.1', '8.2', '8.3', '8.4', '8.5', '9.1', '9.2', '9.3', '9.4', '9.5', '9.6', '9.7', '10.1', '10.2', '10.3', '10.4', '10.5', '10.6', '10.7', '10.8', '11.1', '11.2', '11.3', '11.4', '12.1', '12.2', '12.3', '12.4', '13.1', '13.2', '13.3', '13.4', '13.5', '13.6', '13.7', '14.1', '14.2', '14.3', '14.4', '14.5', '14.6', '14.7', '15.1', '15.2', '15.3', '15.4', '15.5', '15.6', '16.1', '16.2', '16.3', '16.4', '16.5', '16.6', '17.1', '17.2', '17.3', '17.4', '17.5', '17.6', '17.7', '17.8'])

In [46]:
subchapter_to_lgs = dict(zip(metadata_df['Subchapter Number'], metadata_df['Subchapter Learning Objectives']))
set(questions.keys()).difference(set(subchapter_to_lgs.keys()))

set()

In [47]:
dataset = []
for subchapter, question_list in questions.items():
  for question in question_list:
    for learnning_goal in subchapter_to_lgs[subchapter].split('\n'):
      dataset.append([question, learnning_goal])

dataset = pd.DataFrame(data=dataset, columns=['question', 'learning_goal'])
dataset.head()

,question,learning_goal
0,Find the order of magnitude of the following p...,Describe the scope of physics.
1,Find the order of magnitude of the following p...,Calculate the order of magnitude of a quantity.
2,Find the order of magnitude of the following p...,"Compare measurable length, mass, and timescale..."
3,Find the order of magnitude of the following p...,"Describe the relationships among models, theor..."
4,Use the orders of magnitude you found in the p...,Describe the scope of physics.


In [48]:
import seaborn as sns

(dataset['learning_goal'].value_counts() >= 5).sum()

282

In [49]:
value_counts = dataset['learning_goal'].value_counts()
value_counts[value_counts >= 5].mean()

11.53191489361702

In [50]:
value_counts[value_counts >= 5].median()

10.0

In [51]:
def k_shot_sample(data, learning_goal, match=True, k=5):
  if match:
    sample_data = data[data['learning_goal'] == learning_goal]
  else:
    sample_data = data[data['learning_goal'] != learning_goal]
  return sample_data.sample(n=min(k, len(sample_data)))
  
def meta_task(data, k=5):
  # very clunky, but only look at data whose learning goals have enough examples
  data = data[data['learning_goal'].isin(
      data['learning_goal'].value_counts()[data['learning_goal'].value_counts() >= k].index
  )]
  query = np.random.choice(data['question'].unique())
  learning_goal = data[data['question'] == query]['learning_goal'].sample().values[0]
  k_shot_true = k_shot_sample(data[data['question'] != query], learning_goal, match=True, k=k)
  k_shot_false = k_shot_sample(data[data['question'] != query], learning_goal, match=False, k=k)
  return k_shot_true, k_shot_false, query, learning_goal


In [52]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased').to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
def evaluate(n=10, k_shot=5, include_learning_goal=True):
  num_correct = 0
  for _ in range(n):
    k_shot_true, k_shot_false, query, learning_goal = meta_task(dataset, k=k_shot)
    inputs_true = tokenizer(
      [
          learning_goal + ' ' + question
          if include_learning_goal else question
          for learning_goal, question in zip(k_shot_true['learning_goal'].values, k_shot_true['question'].values)
      ], 
      return_tensors='pt', 
      truncation=True, 
      padding=True, 
      max_length=128
    )
    inputs_true = {k: v.to(device) for k, v in inputs_true.items()}
    prototype_true = bert(**inputs_true).pooler_output.mean(dim=0)

    inputs_false = tokenizer(
      [
          learning_goal + ' ' + question
          if include_learning_goal else question
          for learning_goal, question in zip(k_shot_false['learning_goal'].values, k_shot_false['question'].values)
      ], 
      return_tensors='pt', 
      truncation=True, 
      padding=True, 
      max_length=128
    )
    inputs_false = {k: v.to(device) for k, v in inputs_false.items()}
    prototype_false = bert(**inputs_false).pooler_output.mean(dim=0)

    query_input = tokenizer([query], return_tensors='pt', truncation=True, padding=True, max_length=128)
    query_input = {k: v.to(device) for k, v in query_input.items()}
    query_embedding = bert(**query_input).pooler_output[0]

    num_correct += (
        torch.nn.functional.pairwise_distance(
            prototype_true, query_embedding
        ).item() < torch.nn.functional.pairwise_distance(
            prototype_false, query_embedding
        ).item()
    )

  return num_correct / n

In [54]:
evaluate(n=400, k_shot=2)

0.57

In [55]:
evaluate(n=400, k_shot=3)

0.5375

In [56]:
evaluate(n=400, k_shot=5)

0.56

In [57]:
evaluate(n=400, k_shot=2, include_learning_goal=False)

0.5775

In [58]:
evaluate(n=400, k_shot=3, include_learning_goal=False)

0.585

In [59]:
evaluate(n=400, k_shot=5, include_learning_goal=False)

0.6225